$\Large\textbf{Answer 2}$

In [15]:
#Importing the necessary packages
import numpy as np 
from numpy.linalg import cond
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style("whitegrid")
import pandas as pd
from scipy.linalg import sqrtm

In [16]:
#method to find Hessian matrix
def evalf(x):
  assert type(x) is np.ndarray and len(x) == 2
  return (400*x[0]**2 + 0.004*x[1]**4)
     
def evalg(x):
  assert type(x) is np.ndarray and len(x) == 2
  return np.array([800*x[0],0.016*x[1]**3])

def evalh(x): 
  assert type(x) is np.ndarray 
  assert len(x) == 2
  return np.array([[800,0],[0,0.048*x[1]**2]])

#method to find the condition number of any square matrix
#def find_condition_number(A):
 # assert type(A) is np.ndarray
  #assert A.shape[0] == A.shape[1]
  #return cond(A)

In [17]:
#The method defines a way to construct D_k matrix used in scaling the gradient in the modified gradient descent scheme

def compute_D_k(x):
    assert type(x) is np.ndarray
    assert len(x) == 2
    
    elem1 = evalh(x)[0][0]
    elem2 = evalh(x)[1][1]

    return np.diag([1/elem1,1/elem2])

def compute_D_k_newton(x):
    assert type(x) is np.ndarray
    assert len(x) == 2
    
    return np.linalg.inv(evalh(x))
  

In [18]:
# Module to compute the steplength by using the closed-form expression

def compute_steplength_backtracking(x, gradf, alpha_start, rho, gamma): #add appropriate arguments to the function 
    assert type(x) is np.ndarray and len(gradf) == 2 
    assert type(gradf) is np.ndarray and len(gradf) == 2 
    #assert type(alpha_start) is float and alpha_start>=0. 
    assert type(rho) is float and rho>=0.
    assert type(gamma) is float and gamma>=0. 
 
    alpha = alpha_start
    p = -gradf
    while evalf(np.add(x, alpha*p)) > np.add(evalf(x), gamma*alpha*np.dot(gradf.T,p)):
        alpha = rho*alpha

    return alpha

In [19]:
def compute_steplength_backtracking_scaled_direction(x, gradf, D_k, alpha_start, rho, gamma): #add appropriate arguments to the function 
    assert type(x) is np.ndarray and len(gradf) == 2 
    assert type(gradf) is np.ndarray and len(gradf) == 2 
    #assert type(alpha_start) is float and alpha_start>=0. 
    assert type(rho) is float and rho>=0.
    assert type(gamma) is float and gamma>=0. 
    assert type(D_k) is np.ndarray and len(D_k) == 2
    
    alpha = alpha_start
    p = -gradf
    while evalf(np.add(x, alpha*np.dot(D_k,p))) > np.subtract(evalf(x), gamma*alpha*np.dot(np.dot(D_k,gradf), gradf)):
    
        alpha = rho*alpha
    
    return alpha 

In [20]:
# line search type 
CONSTANT_STEP_LENGTH = 1
BACKTRACKING_LINE_SEARCH = 2

# D_k choice type
newton = 3
diagonal_approx = 4

In [30]:
# Code for gradient descent with scaling to find the minimizer without scaling

def find_minimizer_gd(start_x, tol, line_search_type, *args):
    #Input: start_x is a numpy array of size 2, tol denotes the tolerance and is a positive float value
    assert type(start_x) is np.ndarray and len(start_x) == 2 #do not allow arbitrary arguments 
    assert type(tol) is float and tol>=0 

    x = start_x
    g_x = evalg(x)

    #initialization for backtracking line search
    if(line_search_type == BACKTRACKING_LINE_SEARCH):
        alpha_start = args[0]
        rho = args[1]
        gamma = args[2]

    k = 0

    while (np.linalg.norm(g_x) > tol): 
    
        if line_search_type == BACKTRACKING_LINE_SEARCH:
            step_length = compute_steplength_backtracking(x,g_x, alpha_start,rho, gamma) 
        elif line_search_type == CONSTANT_STEP_LENGTH: 
            step_length = 1.0
        else:  
            raise ValueError('Line search type unknown. Please check!')
        
        # Gradient descent steps   
        x = np.subtract(x, np.multiply(step_length,g_x))
        k += 1 
        g_x = evalg(x) 

        #print('iter:',k, ' x:', x, ' f(x):', evalf(x), ' grad at x:', g_x, ' gradient norm:', np.linalg.norm(g_x))
    return x, k, evalf(x) 
 

In [31]:
# Code for gradient descent with scaling to find the minimizer with scaling

def find_minimizer_gdscaling(start_x, tol, line_search_type, D_k_type, *args):
    #Input: start_x is a numpy array of size 2, tol denotes the tolerance and is a positive float value
    assert type(start_x) is np.ndarray and len(start_x) == 2 
    assert type(tol) is float and tol>=0 

    x = start_x
    g_x = evalg(x)
    
    #initialization for backtracking line search
    alpha_start = args[0]
    rho = args[1]
    gamma = args[2]

    k = 0

    while (np.linalg.norm(g_x) > tol): 
        if D_k_type == newton:
            d_k = compute_D_k_newton(x)
        elif D_k_type == diagonal_approx:
            d_k = compute_D_k(x)
        else:
            raise ValueError("Invalid argument.")


        if line_search_type == BACKTRACKING_LINE_SEARCH:
            step_length = compute_steplength_backtracking_scaled_direction(x, g_x, d_k, alpha_start, rho, gamma) 
        elif line_search_type == CONSTANT_STEP_LENGTH: 
            step_length = 1.0
        else:  
            raise ValueError('Line search type unknown. Please check!')
        
        # Gradient descent steps
        x = np.subtract(x, np.multiply(step_length,np.dot(d_k, g_x))) 
        k += 1 
        g_x = evalg(x) 
        #print('iter:',k, ' x:', x, ' f(x):', evalf(x), ' grad at x:', g_x, ' gradient norm:', np.linalg.norm(g_x))


    return x, k, evalf(x)
 

$\Large\textbf{Answer 3}$

In [23]:
my_start_x = np.array([2.0,2.0])
my_tol= 1e-9
alpha_start = 1.0
rho = 0.5
gamma = 0.5

In [24]:
# Gradient descent using Newton's method with constant step length with scaling
print('Results for constant step length')
minimizer, iterations, objective = find_minimizer_gdscaling(my_start_x, my_tol, CONSTANT_STEP_LENGTH, newton, alpha_start, rho, gamma)
print('Minimizer:', minimizer, '\nObjective function value:', objective, '\nIterations taken:', iterations)


# Gradient descent using Newton's method with backtracking line search with scaling
print('Results for backtracking line search')
minimizer, iterations, objective = find_minimizer_gdscaling(my_start_x, my_tol, BACKTRACKING_LINE_SEARCH, newton, alpha_start, rho, gamma)
print('Minimizer:', minimizer, '\nObjective function value:', objective, '\nIterations taken:', iterations)


Results for constant step length
Minimizer: [0.         0.00304488] 
Objective function value: 3.4382653805813626e-13 
Iterations taken: 16
Results for backtracking line search
Minimizer: [0.         0.00304488] 
Objective function value: 3.4382653805813626e-13 
Iterations taken: 16


$\Large\textbf{Observations}$

**Minimizer:**  [0.         0.00304488] 

**Objective function value:**  3.4382653805813626e-13 

**Iterations taken:** 16

*  The above result is same for the both the cases i.e., in newton's method with constant step length and in newton's method with backtracking line search.

*  We can see that the choice of steplength did not affect the convergence of algorithm.

After running the code for few hours the algorithm stops due to very low tolerance so we'll choose greater tolerance say 10^-6.

In [25]:
my_start_x = np.array([2.0,2.0])
my_tol= 1e-6
alpha_start = 1.0
rho = 0.5
gamma = 0.5

In [32]:
# Gradient descent with backtracking line search without scaling
minimizer, iters, objective = find_minimizer_gd(my_start_x, my_tol, BACKTRACKING_LINE_SEARCH, alpha_start, rho, gamma)
print('Minimizer:', minimizer, '\nFinal Objective function value:', objective, '\nIterations taken to terminate:', iters)

Minimizer: [3.53148711e-10 3.91377499e-02] 
Final Objective function value: 9.385197325158433e-09 
Iterations taken to terminate: 5707240


In [26]:
# Gradient descent using backtracking line search with scaling using diagonal matrix
minimizer, iter, objective = find_minimizer_gdscaling(my_start_x, my_tol, BACKTRACKING_LINE_SEARCH, diagonal_approx, alpha_start, rho, gamma)
print('Minimizer:', minimizer ,'\nFinal Objective function value:', objective, '\nIterations taken to terminate:', iter)

Minimizer: [0.         0.03468306] 
Final Objective function value: 5.788014517642641e-09 
Iterations taken to terminate: 10


$\Large\textbf{Observations}$

**For backtracking without scaling:**

Minimizer: [3.53148711e-10 3.91377499e-02] 

Final Objective function value: 9.385197325158433e-09 

Iterations taken to terminate: 5707240

**For backtracking with scaling using diagonal matrix:**

Minimizer: [0.         0.03468306] 

Final Objective function value: 5.788014517642641e-09 

Iterations taken to terminate: 10

When we took lesser tolerance value (1e-9) it took hours long to converge without scaling. But for 1e-6, it took only 10 minutes.

The number of iterations taken by with scaling is is much much lesser than taken by without scaling.